In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import gspread
from gspread_dataframe import set_with_dataframe

In [2]:
gc = gspread.service_account(filename='taller-tarea-4-316303-9047df5b1f74.json')
sh = gc.open_by_key('1FhJn1bBvo3Ph_1YcZLyjooiIwPw0yI1ZBkf85CP-Ozs')

In [3]:
worksheet = sh.get_worksheet(0)

In [4]:
worksheet.clear()

{'spreadsheetId': '1FhJn1bBvo3Ph_1YcZLyjooiIwPw0yI1ZBkf85CP-Ozs',
 'clearedRange': "'Hoja 1'!A1:Z38966"}

In [5]:
COUNTRY_DICT = {"CHILE": "CHL", "ARGENTINA": "ARG",
                "BRASIL": "BRA", "EEUU": "USA", "ALEMANIA": "DEU", "FRANCIA": "FRA"}

xml_chl = requests.get(
    f'http://tarea-4.2021-1.tallerdeintegracion.cl/gho_{COUNTRY_DICT["CHILE"]}.xml')
xml_arg = requests.get(
    f'http://tarea-4.2021-1.tallerdeintegracion.cl/gho_{COUNTRY_DICT["ARGENTINA"]}.xml')
xml_bra = requests.get(
    f'http://tarea-4.2021-1.tallerdeintegracion.cl/gho_{COUNTRY_DICT["BRASIL"]}.xml')
xml_usa = requests.get(
    f'http://tarea-4.2021-1.tallerdeintegracion.cl/gho_{COUNTRY_DICT["EEUU"]}.xml')
xml_deu = requests.get(
    f'http://tarea-4.2021-1.tallerdeintegracion.cl/gho_{COUNTRY_DICT["ALEMANIA"]}.xml')
xml_fra = requests.get(
    f'http://tarea-4.2021-1.tallerdeintegracion.cl/gho_{COUNTRY_DICT["FRANCIA"]}.xml')

In [6]:
etree_chl = ET.fromstring(xml_chl.content)
etree_arg = ET.fromstring(xml_arg.content)
etree_bra = ET.fromstring(xml_bra.content)
etree_usa = ET.fromstring(xml_usa.content)
etree_deu = ET.fromstring(xml_deu.content)
etree_fra = ET.fromstring(xml_fra.content)

In [7]:
all_etree = [etree_chl, etree_arg, etree_bra, etree_usa, etree_deu, etree_fra]

In [8]:
indicator_list = ["Number of deaths", 
                  "Number of infant deaths", "Number of under-five deaths", 
                "Mortality rate for 5-14 year-olds (probability of dying per 1000 children aged 5-14 years)",
                  "Adult mortality rate (probability of dying between 15 and 60 years per 1000 population)",
                   "Estimates of number of homicides",
                   "Crude suicide rates (per 100 000 population)",
                   "Mortality rate attributed to unintentional poisoning (per 100 000 population)",
                   "Number of deaths attributed to non-communicable diseases, by type of disease and sex",
                   "Estimated road traffic death rate (per 100 000 population)",
                   "Estimated number of road traffic deaths",
                   "Mean BMI (kg/m&#xb2;) (crude estimate)",
                   "Mean BMI (kg/m&#xb2;) (age-standardized estimate)",
                   "Prevalence of obesity among adults, BMI &GreaterEqual; 30 (age-standardized estimate) (%)",
                   "Prevalence of obesity among children and adolescents, BMI > +2 standard deviations above the median (crude estimate) (%)",
                   "Prevalence of overweight among adults, BMI &GreaterEqual; 25 (age-standardized estimate) (%)",
                   "Prevalence of overweight among children and adolescents, BMI > +1 standard deviations above the median (crude estimate) (%)",
                   "Prevalence of underweight among adults, BMI < 18.5 (age-standardized estimate) (%)",
                   "Prevalence of thinness among children and adolescents, BMI < -2 standard deviations below the median (crude estimate) (%)",
                   "Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)",
                   "Estimate of daily cigarette smoking prevalence (%)",
                   "Estimate of daily tobacco smoking prevalence (%)",
                   "Estimate of current cigarette smoking prevalence (%)",
                   "Estimate of current tobacco smoking prevalence (%)",
                   "Mean systolic blood pressure (crude estimate)",
                   "Mean fasting blood glucose (mmol/l) (crude estimate)",
                   "Mean Total Cholesterol (crude estimate)"
          ]

In [9]:
def getAllRows(etree_list, indicator_list):
    rows = []
    for tree in etree_list:
        for node in tree:
            indicator = node.find("GHO").text if node.find("GHO") is not None else None
            if indicator in indicator_list:
                country = node.find("COUNTRY").text if node.find("COUNTRY") is not None else None
                sex = node.find("SEX").text if node.find("SEX") is not None else None
                ghecauses = node.find("GHECAUSES").text if node.find("GHECAUSES") is not None else None
                age_group = node.find("AGEGROUP").text if node.find("AGEGROUP") is not None else None
                year = node.find("YEAR").text if node.find("YEAR") is not None else None
                numeric = node.find("Numeric").text if node.find("Numeric") is not None else None
                display = node.find("Display").text if node.find("Display") is not None else None
                low = node.find("Low").text if node.find("Low") is not None else None
                high = node.find("High").text if node.find("High") is not None else None
                rows.append({"GHO": indicator, "COUNTRY": country, "SEX": sex, "GHECAUSES":ghecauses, 
                         "AGEGROUP": age_group, "YEAR": year, "NUMERIC": numeric,
                         "DISPLAY": display, "LOW": low, "HIGH": high})
    return rows

In [10]:
df_columns = ["GHO","COUNTRY", "SEX", "GHECAUSES", "AGEGROUP", "YEAR", "NUMERIC", "DISPLAY", "LOW", "HIGH"]

In [12]:
all_rows = getAllRows(all_etree, indicator_list)

In [13]:
out_pd = pd.DataFrame(all_rows, columns=df_columns)

In [14]:
out_pd

,GHO,COUNTRY,SEX,GHECAUSES,AGEGROUP,YEAR,NUMERIC,DISPLAY,LOW,HIGH
0,Mean BMI (kg/m&#xb2;) (crude estimate),Chile,Male,None,20+ years,1980,24.30000,24.3 [22.4-26.2],22.40000,26.20000
1,Mean BMI (kg/m&#xb2;) (crude estimate),Chile,Male,None,20+ years,1981,24.30000,24.3 [22.6-26.1],22.60000,26.10000
2,Mean BMI (kg/m&#xb2;) (crude estimate),Chile,Male,None,20+ years,1982,24.40000,24.4 [22.7-26.0],22.70000,26.00000
3,Mean BMI (kg/m&#xb2;) (crude estimate),Chile,Male,None,20+ years,1983,24.40000,24.4 [22.9-26.0],22.90000,26.00000
4,Mean BMI (kg/m&#xb2;) (crude estimate),Chile,Male,None,20+ years,1997,25.80000,25.8 [25.0-26.6],25.00000,26.60000
...,...,...,...,...,...,...,...,...,...,...
29178,Mortality rate for 5-14 year-olds (probability...,France,Both sexes,None,None,2000,1.38907,1.4 [1.4-1.4],1.35696,1.42803
29179,Mortality rate for 5-14 year-olds (probability...,France,Both sexes,None,None,2007,0.99582,1.0 [1.0-1.0],0.96213,1.02593
29180,Mortality rate for 5-14 year-olds (probability...,France,Both sexes,None,None,2009,0.93748,0.9 [0.9-1.0],0.91176,0.96172
29181,Mortality rate for 5-14 year-olds (probability...,France,Both sexes,None,None,2016,0.78737,0.8 [0.8-0.8],0.76431,0.81177


In [31]:
new_pd = out_pd[out_pd["GHO"].str.contains(
    "deaths") | out_pd["GHO"].str.contains("mortality") | out_pd["GHO"].str.contains("homicides") | out_pd["GHO"].str.contains("traffic") |
    out_pd["GHO"].str.contains("suicide") | out_pd["GHO"].str.contains("BMI") | out_pd["GHO"].str.contains("Alcohol, recorded") | out_pd["GHO"].str.contains("smoking prevalence") |\
        out_pd["GHO"].str.contains("Mean")]

In [33]:
new_pd

,GHO,COUNTRY,SEX,GHECAUSES,AGEGROUP,YEAR,NUMERIC,DISPLAY,LOW,HIGH
1,"Water, sanitation and hygiene attributable dea...",Chile,None,None,None,2004,1.26000,1,None,None
2,"Water, sanitation and hygiene attributable de...",Chile,None,None,None,2004,0.63000,1,None,None
4,Ambient air pollution attributable deaths per...,Chile,None,None,None,2004,0.19000,0,None,None
5,Household air pollution attributable deaths pe...,Chile,None,None,None,2004,0.00000,0,None,None
9,Ambient air pollution attributable deaths,Chile,None,None,None,2008,3193.00000,3193,None,None
...,...,...,...,...,...,...,...,...,...,...
117786,"Alcohol, recorded per capita (15+) consumption...",France,None,None,None,2016,2.43000,2.43,1.49066,3.31932
117787,"Alcohol, recorded per capita (15+) consumption...",France,None,None,None,2016,11.74000,11.74,8.94130,14.62113
117788,"Alcohol, recorded per capita (15+) consumption...",France,None,None,None,2017,6.79000,6.79,4.05921,9.46414
117789,"Alcohol, recorded per capita (15+) consumption...",France,None,None,None,2018,6.29000,6.29,3.80619,8.83065


In [15]:
set_with_dataframe(worksheet, out_pd)